In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('AdaBins')
sys.path.append('CLIP')

In [2]:
from munch import Munch
from tqdm.auto import tqdm
from torchvision.transforms import ToTensor, Normalize, Resize, Compose
from PIL import Image
import os
import torch
import numpy as np

from datasets import *
from model import Model
from transforms import *
from utils import compute_metrics

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.makedirs('checkpoint', exist_ok=True)

/home/olga/depth_prediction/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model = Model(min_val=1e-3, max_val=180, K=100, norm='linear')
model = model.to(device)
model.load_state_dict(torch.load('checkpoint/model_73930.ckpt'));

In [5]:
model.eval();

In [43]:
IMG_PATH = 'train2017'
DEPTH_PATH = 'COCO_midas_depth'

img_transform = Compose([
    ToTensor(),
    Resize((224,224), interpolation=Image.BILINEAR),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_metrics = Munch(
    rel=0, rms=0, log_10=0, sq_rel=0, rmse_log=0,
    threshold_acc_1=0, threshold_acc_2=0, threshold_acc_3=0)

cnt = 0
for image_name in tqdm(os.listdir(IMG_PATH)):
    cnt += 1
    image_path = os.path.join(IMG_PATH, image_name)
    depth_path = os.path.join(DEPTH_PATH, image_name.replace('.jpg', '.npy'))
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = img_transform(image).unsqueeze(0)

    depth_gt = np.load(depth_path)[np.newaxis, ...]
    with torch.no_grad():
        pred = model(image.to(device))[-1].cpu()
        pred = torch.clip(pred, 1e-3, 180)
        image = torch.flip(image, [-1])
        pred_lr = model(image.to(device))[-1].cpu()
        pred_lr = torch.flip(pred_lr, [-1])
        pred_lr = torch.clip(pred_lr, 1e-3, 180)
        
    final = 0.5 * (pred + pred_lr)
    if final.shape != depth_gt.shape:
        final = nn.functional.interpolate(final, depth_gt.shape[-2:], mode='bilinear', align_corners=True)[0].numpy()
        
    final[final < 1e-3] = 1e-3
    final[final > 180] = 180
    final[np.isinf(final)] = 180
    final[np.isnan(final)] = 1e-3
    
    mask = (depth_gt > 1e-3) * (depth_gt < 180)
    metrics = compute_metrics(depth_gt[mask], final[mask])
    for m in metrics.keys():
        test_metrics[m] += metrics[m]

for m in metrics.keys():
    test_metrics[m] /= cnt

test_metrics

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118287/118287 [4:58:32<00:00,  6.60it/s]


Munch({'rel': 1.1433313165791876, 'rms': 6.757456747567729, 'log_10': 0.17277100761165964, 'sq_rel': 5.192733242787379, 'rmse_log': 0.5352806439334903, 'threshold_acc_1': 0.34421077874153405, 'threshold_acc_2': 0.6807899167752951, 'threshold_acc_3': 0.8751707355257307})